In [1]:
import pandas as pd
import numpy as np
import glob
from random import *
import os
import itertools
import gc
import sklearn
from sklearn.model_selection import train_test_split

In [42]:
class datasets(object):
    seed(10)
    def __init__(self,scale:int):

        
        df = pd.read_csv('../config/patch_info.csv')
        df = df[df[f'{scale}x'] != 0]   # remove those patients with no extracted patches under the very scale
        self.df = df
        self.scale = scale

        # self.H_count = df['TMB_H/L'].value_counts()['H']
        # self.L_count = df['TMB_H/L'].value_counts()['L']
        # self.H_patches_count = df[df['TMB_H/L']=='H'][f'{scale}x'].sum()
        # self.L_patches_count = df[df['TMB_H/L']=='L'][f'{scale}x'].sum()
        # self.H_list = df[df['TMB_H/L']=='H']['dir_uuid']
        # self.L_list = df[df['TMB_H/L']=='L']['dir_uuid']
        
    
                
    
    def __segmentation(self):
        '''
        train:val:train = 0.64:0.16:0.2
        percentage of H / L should be balanced among three sets :set shuffle = True
        '''
        df = self.df
        uuids = df['dir_uuid']
        labels = df['TMB_H/L']

        x_tv,x_test,y_tv,y_test = train_test_split(uuids,labels,train_size=0.8,random_state=10,shuffle=True,stratify=labels)
        
        x_train,x_val,y_train,y_val = train_test_split(x_tv,y_tv,train_size=0.8,random_state=10,shuffle=True,stratify=y_tv)

        df_train = df[df['dir_uuid'].isin(x_train)][['dir_uuid','TMB_H/L',f'{self.scale}x']]
        df_val = df[df['dir_uuid'].isin(x_val)][['dir_uuid','TMB_H/L',f'{self.scale}x']]
        df_test = df[df['dir_uuid'].isin(x_test)][['dir_uuid','TMB_H/L',f'{self.scale}x']]
        
        return df_train,df_val,df_test
        # num_val_H = int(self.H_count * 0.16)
        # num_val_L = int(self.L_count * 0.16)
        # num_test_H = int(self.H_count *0.2)
        # num_test_L = int(self.L_count * 0.2)
        # num_train_H = self.H_count - num_val_H - num_test_H
        # num_train_L = self.L_count - num_val_L - num_test_L

        # target_patches_val_H = int(self.H_patches_count *0.16)
        # target_patches_val_L = int(self.L_patches_count *0.16)
        # target_patches_test_H = int(self.H_patches_count *0.2)
        # target_patches_test_L = int(self.L_patches_count *0.2)
        # target_patches_train_H = self.H_patches_count - target_patches_val_H - target_patches_test_H
        # target_patches_train_L = self.L_patches_count - target_patches_val_L - target_patches_test_L


        
        # dif_test_H = []
        # list_test_H = []
        # dif_test_L = []
        # list_test_L = []
        # for cb_test_H in itertools.combinations(self.H_list,num_test_H):
        #     patch_count = df[df['dir_uuid'].isin(cb_test_H)][f'{self.scale}x'].sum()
        #     print(patch_count)
        #     dif_test_H.append(patch_count - target_patches_test_H)
        #     list_test_H.append(cb_test_H)
        # print(self.H_count,num_test_H)
        
        # result = pd.DataFrame({
        #         'dif':dif_test_H,
        #         'combinations':list_test_H
        # }
        # )
        # print(result)


        # shuffle(self.H_list)
        # shuffle(self.L_list)
        
        # train_list = self.H_list[:int(self.H_len*0.64)] + self.L_list[:int(self.L_len*0.64)]
        # val_list = self.H_list[int(self.H_len*0.64):int(self.H_len*0.8)] + self.L_list[int(self.L_len*0.64):int(self.L_len*0.8)]
        # test_list = self.H_list[int(self.H_len*0.8):]+self.L_list[int(self.L_len*0.8):]
        # shuffle(train_list)
        # shuffle(val_list)
        # shuffle(test_list)
        # new_img_list = train_list+val_list+test_list
        # return train_list,val_list,test_list,new_img_list


    # def __combination_distance(self,df,num_patient,TMB_level,target_patches,it = True):
    #     '''
    #     notice:
    #         violate principle of randomization
    #     args:
    #         df:之前生成的一个Dataframe, 存储有病人的uuid及不同方法倍数下经过extract——patches 及CN操作后得到的patch数量
    #         num_patient:希望得到的一个组合的病人数量, 比如5x病人 L:H = 300:64, testing_set病人中H数量就应该为int(64*0.2)
    #         TMB_level: H / L, 分别取H / L的病人然后拼成一个集
    #         target_patches: 希望得到的这个组合的patch总数, 比如5x病人共有27000张5x图片, 希望testing set中标记为H的patch总数就是27000*(64/364)*0.2
    #         it: 用于对比两种方法的开关
    #     '''

    #     #分别取H / L两组病人,self.H/L_list为该分辨率下patch数不为0的TMB leve为H/L的病人uuid列表
    #     if TMB_level == 'H':
    #         ls = self.H_list  
    #     else:
    #         ls = self.L_list
    #     dif = 1000000  #int，设置dif的基准值，后续在for循环中更新。表示该病人组合的得到的patch数与目标patch数的差距
    #     combinations = ()  #tuple，最优病人组合
        
    #     #只用在第二种方法的一些参数
    #     i = 0 
    #     counts = []
    #     ls_comb = []


    #     for cb_H in itertools.combinations(ls,num_patient):   
    #         '''
    #         cb_H: combinations of patients H
    #         ls:见line 108
    #         num_patient:见line 101
    #         itertools.combinations(ls,num_patient):
    #             generator, 在ls里取num_patient的所有组合,returns a tuple
    #             eg:ls = [0,1,2,3],num_patient = 2,则共有C4,2 = 6种取法:(0,1),(0,2),(0,3),(1,2),(1,3),(2,3)
    #         '''
    #         patch_count = df[df['dir_uuid'].isin(cb_H)][f'{self.scale}x'].sum() #patch count为对这个病人组合的patch数求和得到的patch总数

    #         distance = abs(patch_count-target_patches)  #distance表示patch count与target_patches的绝对距离，距离越短的认为越优
    #         #方法1：一个一个对比，取绝对距离最小的那个
    #         if it:
    #             if distance < dif:
    #                 dif = distance
    #                 combinations = cb_H
    #         #方法2：每个批次取200个，先将200个的patch_count（这里用count）和病人组合（这里用ls_comb）存进列表，然后转换为numpy，取批次最小值，然后每个批次对比
    #         else:
    #             i+=1
    #             counts.append(patch_count)
    #             ls_comb.append(cb_H)
    #             counts_np = np.absolute(np.array(counts)-target_patches)
    #             distance = np.min(counts_np)
    #             index = np.argmin(counts_np)
    #             comb = ls_comb[index]
    #             if i%200 ==0:
    #                 if distance < dif:
    #                     dif = distance
    #                     combinations = comb
    #                 counts = []
    #                 ls_comb = []

    #     return dif,combinations

    # def get(self):
    #     return self.__segmentation()

    '''
    get uuids & labels of different datasets
    '''
    def get_train(self):
        return self.__segmentation()[0]
    
    def get_val(self):
        return self.__segmentation()[1]
    
    def get_test(self):
        return self.__segmentation()[2]
    
    def get_csv(self,dir='../config/data_segmentation_csv'):
        #notice: adjust the path for data saving
        # new_img_list = self.__segmentation()[3]
        # ground_truth = {path:path.split('/')[4] for path in new_img_list}
        
        # df = pd.DataFrame(list(ground_truth.items()))
        # #save ground truth as csv
        
        if not os.path.exists(dir):
            os.mkdir(dir)
        # path_csv = os.path.join(dir,f'{self.scale}X_ground_truth.csv')
#         df.to_csv(path_csv)
        
        #save grouping as pickle
        grouping = {
            'train_list':self.get_train(),
            'val_list': self.get_val(),
            'test_list':self.get_test()
        }
        
        path_grouping = os.path.join(dir,f'{self.scale}X_grouping.npy')
        np.save(path_grouping,grouping)

    def __getattribute__(self, name: str) :
        return object.__getattribute__(self,name)

In [52]:
data_5x = datasets(5)
data_10x = datasets(10)
data_20x = datasets(20)
data_40x = datasets(40)

In [53]:
data_5x.get_csv()
data_10x.get_csv()
data_20x.get_csv()
data_40x.get_csv()

In [7]:
data = np.load('../config/data_segmentation_csv/5X_grouping.npy',allow_pickle=True).item()
data['train_list']

,dir_uuid,TMB_H/L,5x
1,bff29d20-3a8f-4a5d-a2de-0e142390551d,L,265
2,edcf4ae6-c985-40ad-aff4-a0ce31b46aeb,L,229
3,e20c2176-2b57-4cc9-a68a-eb6933bf60b1,L,3
5,70e383bf-91cb-4a65-b12a-950f365c4d62,L,135
6,790d96d7-1191-4cf7-9c78-3e879723afd8,L,305
...,...,...,...
371,278e745d-72e3-4538-8b07-0b9157e605d3,H,6
372,613f9ee2-07d2-4ec7-98b5-335e3eb164cd,H,22
374,05e99737-1738-432d-bf0e-7ca0bbb11c31,L,180
375,f6f1c724-96f3-45ac-ab48-446fca061d23,L,92


一个一个对比花费23.9s（num_patients = 3） 6m47s（num_patients = 4）

按batch对比与一个一个对比的时间花费对比: 

|num_patient|one by one|i=10|i=50|i=100|i=200|i=1000|
|:---:|:---:|:---:|:---:|:---:|:---:|:---:|
|3|23.9|26.3|25.5|24.2|25.7|27.1|
|4|6m47||6min54|6m27s|6m13s|

1.num_patient = 3:i =10 使用26.3s，i=50使用25.5，i=100 使用24.2s，i=200 使用 25.7s，i=500使用24.9s，i=1000使用27.1s